In [630]:
import pandas as pd
import numpy as np
import json

In [631]:
pd.options.display.max_columns = None

In [632]:
final_agency_name_mapping = pd.read_csv('../13_Merge_Table/Final_Agency_Name_Mapping.csv')
final_agency_name_mapping

,Agency_Name,Mapped_Name,Country
0,100 Anniversary Chulalongkorn University,100 Anniversary Chulalongkorn University,Thailand
1,100-year-old Chulalongkorn University,100-year-old Chulalongkorn University,Thailand
2,100th Anniversary Chulalongkorn University,100th Anniversary Chulalongkorn University,Thailand
3,100th Anniversary Chulalongkorn University Fun...,100th Anniversary Chulalongkorn University,Thailand
4,100th Anniversary Chulalongkorn University for...,100th Anniversary Chulalongkorn University,Thailand
...,...,...,...
8330,world class university,Thailand National Organization,Thailand
8331,Åbo Akademi University,Åbo Akademi University,Finland
8332,Österreichische Agentur für Internationale Mob...,Österreichische Agentur für Internationale Mob...,Austria
8333,Российский Фонд Фундаментальных Исследований (...,Российский Фонд Фундаментальных Исследований (...,Russia


In [633]:
target_source = '../../DATA/JSON/PAPAER_SCRAPING.json'
data = []

with open(target_source) as d:
    data = json.load(d)

In [634]:
len(data)

2414

In [635]:
data[0]

{'_id': {'$oid': '6751e8034da435bf323c327b'},
 'index': 0,
 'data': {'Eid': '2-s2.0-85029781991',
  'Year': 2017,
  'idxterms': ['Concretes',
   'Fiber reinforced materials',
   'Fibers',
   'High performance concrete',
   'Shear flow',
   'Experimental program',
   'Fiber contents',
   'High performance fiber reinforced concrete',
   'Main parameters',
   'Shear capacity',
   'Shear reinforcement',
   'Structural behaviors',
   'Ultimate capacity',
   'Reinforced concrete'],
  'authkeyword': ['Fiber content',
   'High performance fiber reinforced concrete (HPFRC)',
   'Shear capacity',
   'Shear reinforcement'],
  'Funding_Agency': [{'name': 'Ratchadapisek Sompoch Endowment Fund',
    'link': None},
   {'name': 'Ministry of Education',
    'link': 'https://www.fundinginstitutional.com/funders/100009950'},
   {'name': 'Chulalongkorn University',
    'link': 'https://www.fundinginstitutional.com/funders/501100002873'}],
  'Cited_Count': 61,
  'Document_type': 'Article'}}

In [636]:
idx_terms_list = []
bucket1 = []
bucket2 = []
valid_data = []

In [637]:
for i in data:
    if (i['data']['Document_type'][-1] == '•'):
        i['data']['Document_type'] = i['data']['Document_type'][:-1]
    
    if (i['data']['Document_type'] == 'Article'):
        bucket1.append(i)

In [638]:
len(bucket1)

1792

In [639]:
for i in bucket1:
    if (len(i['data']['idxterms']) == 0 or (len(i['data']['idxterms']) != 0 and i['data']['idxterms'][0] != '')):
        bucket2.append(i)

In [640]:
len(bucket2)

1717

In [641]:
for i in bucket2:
    if (len(i['data']['authkeyword']) == 0 or (len(i['data']['authkeyword']) != 0 and i['data']['authkeyword'][0] != '')):
        valid_data.append(i)

In [642]:
len(valid_data)

1717

In [643]:
authkeyword_list = []
idxterms_list = []
agency_list = []
agency_list_one_row = []
funding_status_cite_list = []


In [644]:
idxterms_long_list = []

In [645]:
for i in valid_data:
    eid = i['data']['Eid']
    authkeyword = i['data']['authkeyword']
    idxterms = i['data']['idxterms']
    cite_count = i['data']['Cited_Count']
    funding_agency = i['data']['Funding_Agency']

    for i in authkeyword:
        authkeyword_list.append([eid, i])

    for i in idxterms:
        idxterms_list.append([eid, i])

    idx_terms_long_list_this_eid = set()

    for i in idxterms:
        idx_terms_long_list_this_eid.add(i)

    idx_terms_long_list_this_eid = list(idx_terms_long_list_this_eid)

    if (len(idx_terms_long_list_this_eid) != 0):
        idx_terms_long_list_this_eid.sort()

    all_idx_term_this_eid_string = ''

    for i in idx_terms_long_list_this_eid:
        all_idx_term_this_eid_string += " "
        all_idx_term_this_eid_string += i 

    if (all_idx_term_this_eid_string == ''):
        all_idx_term_this_eid_string = '---'

    idxterms_long_list.append([eid, all_idx_term_this_eid_string])

    if(funding_agency == None):
        agency_list_one_row.append([eid, None])

    agency_list_this_data = set()
    for i in funding_agency:
        agency_id = None
        if (i['link'] != None):
            agency_id = i['link'].split('/')[-1]
        if (agency_id != None):
            agency_id = str(agency_id)

        agency_name = i['name']
        agency_name_test = pd.DataFrame([agency_name], columns=['Agency_Name'])
        agency_name_test = agency_name_test.merge(final_agency_name_mapping, how='left', on='Agency_Name')
        mapped_name = agency_name_test['Mapped_Name'].iloc[0]
        # print(agency_name, mapped_name)
        if(type(mapped_name) == type("")):
            agency_list_this_data.add(mapped_name)
        else:
            agency_list_this_data.add(agency_name)
        agency_list.append([eid, i['name'], agency_id])


    agency_list_this_data = list(agency_list_this_data)
    agency_list_this_data.sort()
    agency_list_string = ' '.join(agency_list_this_data)
    if(funding_agency != None):
        agency_list_one_row.append([eid,agency_list_string])
    funding_status_cite_list.append([eid, len(funding_agency), cite_count, (0 if len(funding_agency) == 0 else 1), (0 if cite_count == 0 else 1) ])

In [646]:
idxterms_long_list

[['2-s2.0-85029781991',
  ' Concretes Experimental program Fiber contents Fiber reinforced materials Fibers High performance concrete High performance fiber reinforced concrete Main parameters Reinforced concrete Shear capacity Shear flow Shear reinforcement Structural behaviors Ultimate capacity'],
 ['2-s2.0-85039703338',
  ' A-549 cell line A549 Cells Article CD51 antigen Carcinoma, Non-Small-Cell Lung Cell Line, Tumor Cell Movement Cell Proliferation Cell Survival Dendrobium Dendrobium venustum Dendrobium venustum extract Epithelial-Mesenchymal Transition HaCat cell line Humans Integrins Lung Neoplasms NCI-H460 cell line Phenanthrenes Plant Extracts Rac protein Rho guanine nucleotide binding protein alpha5 integrin antineoplastic activity beta3 integrin cell migration cell motility cell motion cell proliferation cell structure cell survival cell viability chemistry controlled study cytotoxicity down regulation drug effects drug isolation enzyme phosphorylation epithelial mesenchymal

In [647]:
idxterms_long_list_df = pd.DataFrame(idxterms_long_list, columns=['Eid', 'Index_Terms'])
idxterms_long_list_df.to_csv('Scrape_Index_Term.csv', index=False)

In [648]:
idxterms_long_list_df

,Eid,Index_Terms
0,2-s2.0-85029781991,Concretes Experimental program Fiber contents...
1,2-s2.0-85039703338,A-549 cell line A549 Cells Article CD51 antig...
2,2-s2.0-85039924338,---
3,2-s2.0-85039922521,---
4,2-s2.0-85039920890,---
...,...,...
1712,2-s2.0-84964267738,Adult Article Child Cone-Beam Computed Tomogr...
1713,2-s2.0-84922900576,climate change conceptual framework demograph...
1714,2-s2.0-84928152781,Thailand ecotoxicology environmental impact e...
1715,2-s2.0-84930797064,---


In [649]:
idxterms_long_list_df[idxterms_long_list_df['Index_Terms'] == '']

,Eid,Index_Terms


In [650]:
authkeyword_df = pd.DataFrame(authkeyword_list, columns=['Eid', 'Auth_Keyword']).drop_duplicates().dropna()

In [651]:
idxterms_df = pd.DataFrame(idxterms_list, columns=['Eid', 'Index_Terms']).dropna() #ยังไม่ได้ drop duplicate
idxterms_df.groupby('Eid')

In [652]:
funding_status_cite_df = pd.DataFrame(funding_status_cite_list, columns=['Eid', 'Funding_Count', 'Cite-By_Count', 'Has_Funding', 'Has_Cite-By']).drop_duplicates().dropna()
funding_status_cite_df

,Eid,Funding_Count,Cite-By_Count,Has_Funding,Has_Cite-By
0,2-s2.0-85029781991,3,61,1,1
1,2-s2.0-85039703338,0,31,0,1
2,2-s2.0-85039924338,3,4,1,1
3,2-s2.0-85039922521,1,9,1,1
4,2-s2.0-85039920890,3,6,1,1
...,...,...,...,...,...
1712,2-s2.0-84964267738,0,26,0,1
1713,2-s2.0-84922900576,0,1846,0,1
1714,2-s2.0-84928152781,0,12,0,1
1715,2-s2.0-84930797064,0,4,0,1


In [653]:
funding_status_cite_df.to_csv('scrape_funding_cite.csv', index=False)

In [654]:
agency_list

[['2-s2.0-85029781991', 'Ratchadapisek Sompoch Endowment Fund', None],
 ['2-s2.0-85029781991', 'Ministry of Education', '100009950'],
 ['2-s2.0-85029781991', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039924338', 'Ratchadapisek Sompoch Endowment Fund', None],
 ['2-s2.0-85039924338', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039924338', 'Thailand Research Fund', '501100004396'],
 ['2-s2.0-85039922521', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039920890', 'National Research University Project', None],
 ['2-s2.0-85039920890',
  'Office of the Higher Education Commission',
  '100012527'],
 ['2-s2.0-85039920890', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039920812', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039917530', 'CU-ASEAN', None],
 ['2-s2.0-85039917530', 'Ministry of Education', '100009950'],
 ['2-s2.0-85039917530', 'Chulalongkorn University', '501100002873'],
 ['2-s2.0-85039914805', 'Chulalongkorn University'

In [655]:
agency_df = pd.DataFrame(agency_list, columns=['Eid', 'Agency_Name', 'Agency_ID'])
agency_df

,Eid,Agency_Name,Agency_ID
0,2-s2.0-85029781991,Ratchadapisek Sompoch Endowment Fund,None
1,2-s2.0-85029781991,Ministry of Education,100009950
2,2-s2.0-85029781991,Chulalongkorn University,501100002873
3,2-s2.0-85039924338,Ratchadapisek Sompoch Endowment Fund,None
4,2-s2.0-85039924338,Chulalongkorn University,501100002873
...,...,...,...
8084,2-s2.0-84979256587,International Center for Green Biotechnology o...,None
8085,2-s2.0-84979256587,Japan Society for the Promotion of Science,501100001691
8086,2-s2.0-84979256587,"Ministry of Education, Culture, Sports, Scienc...",501100001700
8087,2-s2.0-84929649705,Chulalongkorn University,501100002873


In [656]:
agency_country_data = pd.read_csv('agency_data.csv').drop(columns='Unnamed: 0')
agency_country_data = agency_country_data.rename(columns={'Agency_Name' : 'Agency_Name_From_Data'})
agency_country_data['Agency_ID'] = agency_country_data['Agency_ID'].apply(lambda x : str(x))

In [657]:
agency_df = agency_df.merge(agency_country_data, how='left', on='Agency_ID')
agency_df = agency_df.drop(columns='Agency_Name_From_Data')

agency_df

,Eid,Agency_Name,Agency_ID,Agency_Country
0,2-s2.0-85029781991,Ratchadapisek Sompoch Endowment Fund,None,NaN
1,2-s2.0-85029781991,Ministry of Education,100009950,Thailand
2,2-s2.0-85029781991,Chulalongkorn University,501100002873,Thailand
3,2-s2.0-85039924338,Ratchadapisek Sompoch Endowment Fund,None,NaN
4,2-s2.0-85039924338,Chulalongkorn University,501100002873,Thailand
...,...,...,...,...
8084,2-s2.0-84979256587,International Center for Green Biotechnology o...,None,NaN
8085,2-s2.0-84979256587,Japan Society for the Promotion of Science,501100001691,Japan
8086,2-s2.0-84979256587,"Ministry of Education, Culture, Sports, Scienc...",501100001700,Japan
8087,2-s2.0-84929649705,Chulalongkorn University,501100002873,Thailand


In [658]:
agency_df[agency_df['Agency_ID'].isna()]['Agency_Name'].drop_duplicates().to_csv('funding_name_list_scrape.csv', index=False)

In [659]:
agency_list_one_df = pd.DataFrame(agency_list_one_row, columns=['Eid', 'Agency_Name'])
agency_list_one_df.to_csv('funding_field_agency_one_row_scrape.csv', index=False)

In [660]:
agency_list_one_df[agency_list_one_df['Agency_Name'] == '']

,Eid,Agency_Name
1,2-s2.0-85039703338,
6,2-s2.0-85039922690,
10,2-s2.0-85039915342,
12,2-s2.0-85039910610,
13,2-s2.0-85039907641,
...,...,...
1710,2-s2.0-84971373699,
1712,2-s2.0-84964267738,
1713,2-s2.0-84922900576,
1714,2-s2.0-84928152781,


In [661]:
agency_list_one_df.isna().sum()

Eid            0
Agency_Name    0
dtype: int64

In [662]:
agency_df.to_csv('funding_list_by_eid_scrape.csv', index=False)